<a href="https://colab.research.google.com/github/calm-ixia/SDManualGUI/blob/main/SDManualGUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. ライブラリ導入

In [ ]:
# Stable Diffusion パッケージのインストール 
# diffusersの公式リポジトリを参照 https://github.com/huggingface/diffusers
# アカウント登録込みの作業フローは https://programmingforever.hatenablog.com/entry/2022/09/21/174953 も参考
!pip install --upgrade diffusers transformers scipy ftfy
!pip install accelerate

#version 7.7.1 < latest 8.0.4
from ipywidgets import widgets, Layout
from PIL import Image, ImageChops
from PIL.PngImagePlugin import PngInfo
import io

import torch
import diffusers
from diffusers import StableDiffusionPipeline, KarrasVePipeline, ScoreSdeVePipeline
from diffusers.schedulers import *
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker

import os
import math
import time

# xformersには未対応
"""
%pip install xformers==0.0.16rc425
# https://note.com/npaka/n/n5fd3d8ecf1e6
# %pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# http://cedro3.com/ai/dream-booth/
#%pip install -q https://github.com/OMGhozlan/xformers_colab/releases/download/1.0.0/xformers-0.0.15.dev0+cu11.2-cp38-cp38-linux_x86_64.whl
"""

"""
Unstable Diffusion (七師氏のStable Diffusion改造)
  https://github.com/nanashi161382/unstable_diffusion
  https://note.com/tomo161382/n/nff2aa749c48f
"""
!wget 'https://raw.githubusercontent.com/nanashi161382/unstable_diffusion/main/pipeline_layered_diffusion.py' -O /content/unstable_diffusion.py

import unstable_diffusion


# Long Prompt Weighting Stable Diffusion (SkyTNT氏) 
# https://github.com/huggingface/diffusers/tree/main/examples/community#long-prompt-weighting-stable-diffusion
# community pipeline から オリジナルの lpw_stable_diffusion を直接利用する方法に変更した。以下の記述は参考として残す

#if using_lpw_stable_diffusion_mod:
#  # huggingfaceのAlanB氏のリポジトリにある、StableDiffusionLongPromptWeightingPipeline をインポート
#  # https://huggingface.co/docs/hub/models-downloading
#  %pip install huggingface_hub
#  from huggingface_hub import hf_hub_url, hf_hub_download
#  REPO_ID = "AlanB/lpw_stable_diffusion_mod"
#  FILENAME = "pipeline.py"
#  cache = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
#!ln -s {cache} lpw_stable_diffusion_mod.py


# （必要時）割り当てられたGPUの確認

In [ ]:
#@markdown 割り当てられたGPUの確認
!nvidia-smi

#@markdown Googleドライブのマウント
"""
from google.colab import drive
drive.mount('/content/drive/')
"""

# （必要時）Hugging Face へのログイン
Stable Diffusion 2.0, 2.1 では不要。runwayml-v1.5モデルの利用には利用同意とHugging Faceへのログインが要

In [ ]:
#@title 利用同意が必要なモデルを利用する際に、Hugging Face に手動でログイン
# その際に、アクセストークンをnotebookに保存せずに起動させるため、手動ログインを採用
#@markdown Hugging Faceへのログインとは別に、利用モデルの利用規約を読んで同意(Agree and access repository)すること。
#@markdown https://zenn.dev/razokulover/scraps/b4f639228ab305

!huggingface-cli login


#2. GUI実装

In [ ]:
#@title ユーティリティ

# 共通の処理

# バッチ1回で生成した個々の画像オブジェクトと、バッチ1回分のgrid画像オブジェクトを生成
def create_batch_image(outImages, width, height, grid_column_count):
  #gridを生成
  image_count = len(outImages)
  grid_whole_width = width * grid_column_count
  # バッチ1回ごとにグリッドの列数×行数の矩形を作る。余りが出る場合は黒塗りになる
  grid_whole_height = height * math.ceil(image_count/grid_column_count)
  grid_image = Image.new('RGB', size=(grid_whole_width, grid_whole_height) )

  #画像ファイルを保存
  # バッチ1回での行数
  batch_rows = 1 + ((image_count-1)//grid_column_count)
  for batch_index,image in enumerate(outImages):
    # gridの所定位置にペースト
    paste_col_index = batch_index % grid_column_count
    paste_row_index = batch_index//grid_column_count
    grid_image.paste(image, box=(width*paste_col_index, height*paste_row_index) )
  
  return grid_image

# 画像をファイルに保存
def save_image(image, fp, parameter_text):
  # 画像のタグ情報を生成
  grid_info = PngInfo()
  grid_info.add_text("parameters", parameter_text)
  # gridをファイルに保存
  image.save(fp, format="png", pnginfo=grid_info)

# サムネイルを作成、画像を縮小
def create_thumbnail(image, resize=0.125):
  return image.resize(size=(int(image.width*resize),int(image.height*resize)) )

"""
widgets.FileUpload から得られる値は、
{filename: {"metadata":{}, "content":rawdata} } の構造を持つdict
"""
def update_pnginfo(new_value):
  for filename, value in new_value.items():
    metadata = value["metadata"]

    #rawデータを取得
    data = value["content"]
    #画像情報取得用にPILのライブラリで画像を開く
    img = Image.open(io.BytesIO(data))
    #画像情報を取得
    parameter_text = img.info["parameters"]
    return (data, parameter_text)

def fileUploader_load_png(new_value):
  for filename, value in new_value.items():
    #print(value)
    #print(list(value.keys()))
    metadata = value["metadata"]
    #print(metadata)

    #rawデータを取得(bytesオブジェクト)
    data = value["content"]
    return data

def parse_parameter_text(parameter_text):
  param_dict = dict()

  negative_prompt_begin = parameter_text.find("Negative prompt: ")
  # "Negative prompt:" の直前までが通常のプロンプト
  prompt = parameter_text[0:negative_prompt_begin]
  param_dict['Prompt'] = prompt
  other_str = parameter_text[negative_prompt_begin:len(parameter_text)]

  # "Steps:" の直前までがネガティブプロンプト
  step_begin = parameter_text.find("Steps:")
  negative_prompt_pair = parameter_text[negative_prompt_begin:step_begin]
  negative_prompt = negative_prompt_pair.replace('Negative prompt: ', '', 1)
  param_dict['Negative prompt'] = negative_prompt

  # "foo: **, " の形式で設定パラメータが続く
  param_dict_str = parameter_text[step_begin:len(parameter_text)]
  param_pairs = param_dict_str.split(',')
  for pair in param_pairs:
    key,value = pair.split(':')
    # ,:の区切りで分割するだけだと先頭や末尾に空白が残るので、それらを除去
    # 'Euler a' のように語中に空白があるケースもあるので、replace()ではなくstrip()を使用
    param_key = key.strip()
    param_value = value.strip()
    param_dict[param_key] =param_value
  
  # 必要に応じて数値文字列を数値に変換
  conv = {
    'Prompt': lambda x: x,
    'Negative prompt': lambda x: x,
    'Steps': lambda x: int(x),
    'Sampler': lambda x: x, # 文字列をそのまま取得
    'CFG scale': lambda x: float(x),
    'Seed': lambda x: int(x),
    # Sizeをタプルに変換。 "Size: 512x512" のように記述されている
    'Size': lambda x: tuple([int(s) for s in x.split('x')]),
    'Denoising strength': lambda x: float(x),
    'Masked content': lambda x: x,
  }

  result = dict()
  for key,val in param_dict.items():
    if key in conv:
      result[key] = conv[key](val)
  
  return result


In [ ]:
#@title Long Prompt Weighting

# ロード済みのLong Prompt Weighting Pipelineを用いてtext embeddingを生成
# Pipelineの_encode_prompt()だけ使用

from typing import Callable, List, Optional, Union
from diffusers import DiffusionPipeline

class LPWEncoding(unstable_diffusion.StandardEncoding):
  def __init__(self,
    lpw_pipe, #あらかじめモデルをロードしておく
    num_images_per_prompt: Optional[int] = 1,
    max_embeddings_multiples: Optional[int] = 3
  ):
    self.pipe = lpw_pipe
    self.num_images_per_prompt = num_images_per_prompt
    self.max_embeddings_multiples = max_embeddings_multiples

  def Encode(self, text_model: unstable_diffusion.TextModel, text: str):
      # text_modelは不使用。引数の仕様を合わせるだけのプレースホルダ

      # 与えらえたpromptだけをencodeする
      return self.pipe._encode_prompt(
        prompt=text,
        device=self.pipe._execution_device,
        num_images_per_prompt=self.num_images_per_prompt,
        do_classifier_free_guidance=False, # promptのembeddingだけ欲しい
        negative_prompt=None,
        max_embeddings_multiples=self.max_embeddings_multiples,
      )


In [ ]:
#@title 画像生成モジュール用のデータコンテナ

import copy

class GeneratedFileInfo:
  def __init__(self, prompt="", negative_prompt="",
    scheduler_name="", step_count=0, guidance_scale=0, seed=0, width=0, height=0,
    batch_size=0, batch_count=0, grid_column_count=0, enable_safety_checker=True, eta=0.0, resize=0.25, extra_param = {}
  ):
    self.path = ""
    self.thumbnailBytes: bytes = None
    self.imageBytes: bytes = None
    # set parameters
    self.prompt = prompt
    self.negative_prompt = negative_prompt
    self.scheduler_name = scheduler_name
    self.step_count = step_count
    self.guidance_scale = guidance_scale
    self.seed = seed
    self.width = width
    self.height = height
    self.batch_size = batch_size
    self.batch_count = batch_count
    self.grid_column_count = grid_column_count
    self.enable_safety_checker = enable_safety_checker
    self.eta = eta
    self.resize = resize
    self.extra_param = extra_param

  def clone(self):
    cloned = self.clone_light()
    cloned.extra_param = copy.deepcopy(self.extra_param)
    return cloned

  #imageBytes, thumbnailBytes, extra_paramはコピーしないので必要に応じてコピーすること
  def clone_light(self):
    cloned = GeneratedFileInfo()
    cloned.path = copy.deepcopy(self.path)
    # set parameters
    cloned.prompt = copy.deepcopy(self.prompt)
    cloned.negative_prompt = copy.deepcopy(self.negative_prompt)
    cloned.scheduler_name = copy.deepcopy(self.scheduler_name)
    cloned.step_count = copy.deepcopy(self.step_count)
    cloned.guidance_scale = copy.deepcopy(self.guidance_scale)
    cloned.seed = copy.deepcopy(self.seed)
    cloned.width = copy.deepcopy(self.width)
    cloned.height = copy.deepcopy(self.height)
    cloned.batch_size = copy.deepcopy(self.batch_size)
    cloned.batch_count = copy.deepcopy(self.batch_count)
    cloned.grid_column_count = copy.deepcopy(self.grid_column_count)
    cloned.enable_safety_checker = copy.deepcopy(self.enable_safety_checker)
    cloned.eta = copy.deepcopy(self.eta)
    cloned.resize = copy.deepcopy(self.resize)
    return cloned
    
  # 埋め込み用のパラメータ文字列を生成
  def create_parameter_text(self):
  #    f"Steps: {step_count}, Sampler: {scheduler_name}, CFG scale: {guidance_scale}, Seed: {seed}, Size: {width}x{height}, Model hash: {a2a802b2}",
    param = ", ".join([
      f"Steps: {self.step_count}",
      f"Sampler: {self.scheduler_name}",
      f"CFG scale: {self.guidance_scale}",
      f"Seed: {self.seed}",
      f"Size: {self.width}x{self.height}",
    ])
    if 'strength' in self.extra_param:
      param += f", Denoising strength: {self.extra_param['strength']}"
    if 'latents' in self.extra_param:
      param += f", Masked content: {self.extra_param['latents']}"

    return "\n".join([
      self.prompt,
      f"Negative prompt: {self.negative_prompt}",
      param
    ])

  @classmethod
  def from_parameter_text(cls, parameter_text):
    param_dict = parse_parameter_text(parameter_text)
    extra_param = {}
    if 'Denoising strength' in param_dict:
      extra_param['strength'] = param_dict['Denoising strength']
    if 'Masked content' in param_dict:
      extra_param['latents'] = param_dict['Masked content']

    return GeneratedFileInfo(
      prompt = param_dict['Prompt'],
      negative_prompt = param_dict['Negative prompt'],
      scheduler_name = param_dict['Sampler'],
      step_count = param_dict['Steps'],
      guidance_scale = param_dict['CFG scale'],
      seed = param_dict['Seed'],
      width = param_dict['Size'][0],
      height = param_dict['Size'][1],
      #batch_size = param_dict[''],
      #batch_count = param_dict[''],
      #grid_column_count = param_dict[''],
      #enable_safety_checker = param_dict[''],
      #eta = param_dict[''],
      extra_param = extra_param
    )

  def save_image(self, image, filepath):
    parameter_text = self.create_parameter_text()
    save_image(image, filepath, parameter_text)
    with io.BytesIO() as buf:
      save_image(image, buf, parameter_text)
      self.imageBytes = buf.getvalue()

  def save_thumbnail(self, image, resize):
    parameter_text = self.create_parameter_text()
    thumbnail = image.resize(size=(int(image.width*resize),int(image.height*resize)) )
    with io.BytesIO() as buf:
      save_image(thumbnail, buf, parameter_text)
      self.thumbnailBytes = buf.getvalue()



In [ ]:
from transformers.modeling_utils import init_empty_weights
#@title 画像生成モジュール

class BackEnd:
  def __init__(self):
    # LayeredDiffusionPipelineのConnect()による設定ではモデルのリビジョン違い等に対応できないため、手動でPipelineをセットアップ
    self.pipe: StableDiffusionPipeline = None
    self.pipeBuilder: unstable_diffusion.LayeredDiffusionPipeline = None
    self.model = ""
    self.revision = ""
    self.device_to = ""
    self.attention_slicing = False
    self.safety_checker = False
    self.xformers_memory_efficient_attention = False
    # ファイル名につかう時間文字列のフォーマット
    self.timestr_format="%H%M%S_%y%b%d"
    # 更新するペイン
    self.setupPane = None
    self.txt2imgPane = None
    self.img2imgPane = None
    self.inpaintPane = None
    self.pnginfoPane = None
    
    # サンプリング手法（ノイズ除去スケジューラ）
    # https://huggingface.co/docs/diffusers/api/schedulers#implemented-schedulers
    # diffuserでは PNDM Scheduler がデフォルト
    # https://huggingface.co/blog/stable_diffusion
    # https://torch.classcat.com/2022/11/13/huggingface-blog-stable-diffusion/

    #TODO: Karrasタイムステップを用いる手法のSchedulerを実装。特にk-DPM++2M Karras
    self.schedulerDict = {
        "DPM++ 2M": {"SchedulerClass":DPMSolverMultistepScheduler, "desc":"Multistep DPM-Solver++ (DPM:diffusion probabilistic models)"},
        "Euler a": {"SchedulerClass":EulerAncestralDiscreteScheduler, "desc":"Euler Ancestral scheduler  aka k-euler-a"},
        "Euler": {"SchedulerClass":EulerDiscreteScheduler, "desc":"Euler scheduler  aka k-euler"},
        "DPM": {"SchedulerClass":DPMSolverMultistepScheduler, "desc":"Multistep DPM-Solver (DPM:diffusion probabilistic models)"},
        "DPM++ 2S": {"SchedulerClass":DPMSolverSinglestepScheduler , "desc":"Singlestep DPM-Solver++ (DPM:diffusion probabilistic models)"},
        "DDIM": {"SchedulerClass":DDIMScheduler, "desc":"Denoising diffusion implicit models"},
        "DDPM": {"SchedulerClass":DDPMScheduler, "desc":"Denoising diffusion probabilistic models"},
        "PNDM": {"SchedulerClass":PNDMScheduler, "desc":"Pseudo numerical methods for diffusion models  runwayMLモデルのデフォルト"},
        "IPNDM": {"SchedulerClass":IPNDMScheduler, "desc":"Improved Pseudo numerical methods for diffusion models"},
        "LMS": {"SchedulerClass":LMSDiscreteScheduler, "desc":"Linear multistep scheduler for discrete beta schedules  aka k-LMS"},
        "Heun 1": {"SchedulerClass":HeunDiscreteScheduler, "desc":"k-diffusion, Algorithm 1 of Karras et. al."},
        "KDPM2": {"SchedulerClass":KDPM2DiscreteScheduler, "desc":"DPM 2, log interpolation"},
        "KDPM2 a": {"SchedulerClass":KDPM2AncestralDiscreteScheduler, "desc":"KDPM2 Ancestral"},
        "KarrasVe": {"PipelineClass":KarrasVePipeline, "desc":"Variance exploding, stochastic sampling from Karras et. al"},
        "ScoreSdeVe": {"PipelineClass":ScoreSdeVePipeline, "desc":"variance exploding stochastic differential equation (VE-SDE) scheduler"},
        #"ScoreSdeVp":  {"PipelineClass":, "desc":"variance preserving stochastic differential equation (VP-SDE) scheduler"},
    }

  def setup(self, model, revision, device_to, output_dir, default_width, default_height, attention_slicing, xformers_memory_efficient_attention, safety_checker):
    self.model = model # "stabilityai" などのリポジトリ名
    self.revision = revision # "fp16" などのリビジョン名
    self.device_to = device_to
    self.attention_slicing = attention_slicing
    self.xformers_memory_efficient_attention = xformers_memory_efficient_attention #未対応、メンバのみ用意
    self.safety_checker = safety_checker
    self.safety_checker_obj = None
    self.output_dir = output_dir

    #ディレクトリを生成しておく
    os.makedirs(self.output_dir, exist_ok=True)

    # 画像生成に用いるスケジューラ（sampler）をあらかじめ生成しておく
    #  It is deprecated to pass a pretrained model name or path to `from_config`.
    #  If you were trying to load a scheduler, please use (Scheduluerクラス名).from_pretrained(...) instead. 
    # https://note.com/npaka/n/n5fd3d8ecf1e6 を参照した
    for (k,v) in self.schedulerDict.items():
      klass = self.schedulerDict[k]
      if "SchedulerClass" in klass:
        klass["scheduler"] = klass["SchedulerClass"].from_pretrained(model, subfolder="scheduler")
    
    #txt2imgのサンプラーリストとwidth,height初期値を更新
    self.txt2imgPane.set_param( list(self.schedulerDict.keys()), default_width, default_height )
    self.img2imgPane.set_param( list(self.schedulerDict.keys()), default_width, default_height )
    self.inpaintPane.set_param( list(self.schedulerDict.keys()), default_width, default_height )
    
    # construct a pipeline
    # TODO:別のパイプラインを指定できるようにする（img2img, inpaint, KarrasVeなど）

    self.pipe = StableDiffusionPipeline.from_pretrained(
      self.model,
      custom_pipeline="lpw_stable_diffusion",
      torch_dtype=torch.float16,
      revision=self.revision,
    ).to(self.device_to)

    # safety_checkerを退避しておく    
    self.safety_checker_obj = self.pipe.safety_checker
    # ロード済みのcomponentsを持っておき、別のパイプラインに切り替える時に再利用する
    self.components = self.pipe.components

    # 所要時間が増える代わりにGPUメモリ使用量を抑える
    if self.attention_slicing:
      self.pipe.enable_attention_slicing()
    # xformersによるメモリ軽減を図る
    #if self.xformers_memory_efficient_attention:
      #self.pipe.enable_xformers_memory_efficient_attention()

    # LayeredDiffusionPipelineのConnect()による設定ではモデルのリビジョン違い等に対応できないため、手動でセットアップ
    self.pipeBuilder = unstable_diffusion.LayeredDiffusionPipeline()
    #TODO: publicメソッドがあればそちらを使用する
    self.pipeBuilder._dataset = f"{self.model}/{self.revision}"
    self.pipeBuilder._SetPipeline(self.pipe)

    #テキストエンコーダをセット
    #self.textEncoder = unstable_diffusion.ShiftEncoding(rotate=False, reverse=False)
    self.textEncoder = LPWEncoding(self.pipe)

  def word_count(self, prompt):
    untruncated_ids = self.pipe.tokenizer(prompt, padding="longest", return_tensors="pt").input_ids
    return untruncated_ids.shape[-1]

  def max_word_count(self):
    multiple = 1
    if hasattr(self.textEncoder, "max_embeddings_multiples"):
      multiple = self.textEncoder.max_embeddings_multiples
    return multiple * (self.pipe.tokenizer.model_max_length-2)+2 #先頭と末尾の2wordを引いてから倍数をかける

  # Masked Content の "latent nothing"を実装（未完成）
  # TODO: 定数だと更新がかからない（勾配がつかないため？）
  def create_latent_nothing(self, info, value=0.5):
    # https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py から抜粋・変更
    device = self.pipe.device
    dtype = self.pipe.text_encoder.dtype
    num_channels_latents = self.pipe.unet.in_channels
    batch_size = info.batch_size * 1 # num_images_per_prompt = 1 as default
    shape = (batch_size, num_channels_latents, info.height // self.pipe.vae_scale_factor, info.width // self.pipe.vae_scale_factor)
    return torch.full(shape, value, device=device, dtype=dtype).to(self.device_to)

  # Masked Content の "latent noise"を実装
  def create_latent_noise(self, info, generator):
    # https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py から抜粋・変更
    device = self.pipe.device
    dtype = self.pipe.text_encoder.dtype
    num_channels_latents = self.pipe.unet.in_channels
    batch_size = info.batch_size * 1 # num_images_per_prompt = 1 as default
    shape = (batch_size, num_channels_latents, info.height // self.pipe.vae_scale_factor, info.width // self.pipe.vae_scale_factor)
    return torch.randn(shape, device=device, dtype=dtype, generator=generator).to(self.device_to)


  """
  inpaint, img2img, txt2imgの画像生成で使う初期データを決める
  Stable Diffusionにおいてはいずれの機能も手順は同じで、初期データが違う。
  本実装では、img2img, txt2imgはinpaint(InpaintLegacy)の特殊化としている。
  img2img, txt2imgの方が処理が分かりやすいため先に記述。
  """
  def img2img(self, info):
    # 画像全体を初期データとする
    image = info.extra_param["image"]
    strength = info.extra_param["strength"]
    init_latent = unstable_diffusion.ByImage(image=image, strength=strength)
    # layer: 初期データに対するデノイズ指定のレイヤー。プロンプトやCFG scaleの指定はこちらで行う
    # プロンプトのレイヤは1枚だけ
    layer = unstable_diffusion.Layer(
      prompt=info.prompt,
      negative_prompt=info.negative_prompt,
      cfg_scale=info.guidance_scale,
      strength=1.0,
    )
    return init_latent, layer

  def txt2img(self, info):
    # latent noiseを初期データとする
    # txt2imgではRandomlyを使ってもGUI側でのlatent noiseとほぼ同一の画像となる
    #（inpaintでRandomlyを使うとAUTOMATIC1111版に近い画像が出ない。inpaint()での実装を参照）

    # init_latent = unstable_diffusion.Randomly(symmetric=False, strength=1.0)
    init_latent = None # Unstable Diffusionでの修正を反映。initなしの場合はRandomlyが使用される
    # プロンプトのレイヤは1枚だけ
    layer = unstable_diffusion.Layer(
      prompt=info.prompt,
      negative_prompt=info.negative_prompt,
      cfg_scale=info.guidance_scale,
      strength=1.0,
    )
    return init_latent, layer

  # inpaintでの設定。マスク適用の設定により処理を分岐
  def inpaint(self, info):

    # 初期ノイズの種類を設定。
    # ノイズの種類に対してマスクを適用したものが初期latents
    """
    fill: ピクセルのgray( RGB=(128,128,128) )
    original: imageの変換後のノイズをそのまま使う。InpaintPipelineLegacyでimageを与えたときのデフォルト値
    latent noise: 潜在空間上でのランダムノイズ
    latent nothing: 潜在空間上でのzeros
    mode channel: 独自実装。マスク範囲の画素の最頻値を使う。チャネルごとに最頻値を取った値をRGB値とする
    top image: 独自実装。指定した画像をマスクで切り抜いて上(top)に上書き
    random pixel: 独自実装。ピクセル上でのランダムノイズ
    """
    image = info.extra_param["image"]
    mask = info.extra_param["mask_image"]
    strength = info.extra_param["strength"]
    latents_type = info.extra_param["latents"]

    # init_latent: 初期データの指定。[Initializer] も可能で、
    # init_latent[0] に対し、init_latent[1]から順に複数のInitializerを、それぞれ指定したマスクをかけて重ねることが可能

    # init_latentのデフォルト値。imageの変換後のノイズをそのまま使う（originalに相当）
    # strength (Denoise strength): 拡散の順過程でノイズをかける回数の割合[0.0-1.0]。
    # 回数 = numstep * strength、回数が多いほど初期状態がよりノイズに近づき、元の画像とはより異なる画像が生成される。
    latent_image = unstable_diffusion.ByImage(image=image, strength=strength)
    init_latent = latent_image

    # latentsの種別を取得
    if latents_type == "original":
      pass
    elif latents_type == "fill":
      pass
    elif latents_type == "mode channel":
      pass
    
    elif latents_type == "latent noise":
      #マスクして上書き
      # AUTOMATIC1111版からの再現性のため、GUI側でlatent noiseを生成する
      #latent_noise = unstable_diffusion.Randomly(symmetric=False, mask_by=mask, strength=strength)
      latent_noise = unstable_diffusion.ByLatents(latents=self.create_latent_noise(info, self.pipeBuilder.scheduler.generator()), mask_by=mask, strength=strength)
      init_latent = [latent_image, latent_noise]

    elif latents_type == "latent nothing":
      #マスクして上書き
      latent_nothing = unstable_diffusion.ByLatents(latents=self.create_latent_nothing(info), mask_by=mask, strength=strength)
      init_latent = [latent_image, latent_nothing]

    elif latents_type == "randomly":
      # Unstable Diffusion側で latent noise を生成。AUTOMATIC1111版とは得られる画像が異なる
      # symmetric: 対称的なノイズにするかどうか。Trueでだいたい対称的になる
      # TODO: symmetricのチェックボックスの対応
      random_pixel = unstable_diffusion.Randomly(symmetric=False, mask_by=mask, strength=strength)
      init_latent = [latent_image, random_pixel]

    else:
      #未指定＝デフォルト値
      pass

    # layer: 初期データに対するデノイズ指定のレイヤー。プロンプトやCFG scaleの指定はこちらで行う
    layer = unstable_diffusion.Layer(
      prompt=info.prompt,
      negative_prompt=info.negative_prompt,
      cfg_scale=info.guidance_scale,
      strength=1.0, #TODO
      mask_by = mask,
    )
    return init_latent, layer


  # 生成した画像ファイル情報を返す
  # info.seed == -1 （シード値を自動決定）を与えた時は、自動決定されたシード値をinfo.seedにも設定する
  # seed_index: seedに加算、batch_count内で何回目か
  def generate_image_progressive(self, info, pipeMethod):
    # パイプラインを切り替える
    # TODO? : "KarrasVe"などPipelineとSchedulerが不可分な場合の対応
    if "PipelineClass" in self.schedulerDict[info.scheduler_name]:
      pipelineClass = self.schedulerDict[info.scheduler_name]["PipelineClass"]
      if issubclass(pipelineClass, StableDiffusionPipeline) and pipelineClass != type(self.pipe):
        self.pipe = pipelineClass(self.components)
        self.pipeBuilder.pipe = self.pipe
    else:
      # 切り替えない
      self.pipe.scheduler = self.schedulerDict[info.scheduler_name]["scheduler"]

    # unstable_diffusion側のschedulerを設定
    sche = unstable_diffusion.Scheduler()
    sche.scheduler = self.pipe.scheduler
    sche.have_max_timesteps = False
    sche._rand_seed = None
    sche._generator = None
    self.pipeBuilder.scheduler = sche

    #シードをセット
    if(info.seed < 0):
      #シードを得る
      self.pipeBuilder._ResetGenerator(None)
      info.seed = self.pipeBuilder.GetRandSeed()
    self.pipeBuilder._ResetGenerator(info.seed)

    # 設定をパイプラインに反映
    prompt_array = [info.prompt] * info.batch_size
    negative_prompt_array = [info.negative_prompt] * info.batch_size

    if self.safety_checker == True and self.pipe.safety_checker == None:
      self.pipe.safety_checker = self.safety_checker_obj
    else:
      self.pipe.safety_checker = None

    # 機能に対応した初期データとレイヤ構造を得る
    init_latent, layer = pipeMethod(info)

    # initialize:初期データの指定。initialize=[Initializer] も可能で、initialize[0] に対し、initialize[1]から順に複数のinitを、
    # それぞれ指定したマスクをかけて重ねることが可能。（InitializerList.Merge()が使用される）
    # rmm_dump: デバッグ用のレイヤごと出力
    outImage, rmm_dump = self.pipeBuilder(
        num_steps = info.step_count,
        size = (info.width, info.height),
        default_encoding = self.textEncoder,
        rand_seed = info.seed,
        eta = info.eta,
        initialize = init_latent, #上記注
        iterate = layer,
    ) #TODO: 返ってくるのは1バッチ分だけ？
    
    timestr=time.strftime(self.timestr_format)
    if info.batch_size > 1:
      # 1バッチずつファイルに保存
      filepath = os.path.join(self.output_dir, f"batch_{timestr}_seed{info.seed}.png")
      grid_image = create_batch_image(outImage, info.width, info.height, info.grid_column_count)
      info.save_image(grid_image, filepath)
      info.save_thumbnail(grid_image, info.resize)
    else:
      # 1回分のファイルを保存
      filepath = os.path.join(self.output_dir, f"{timestr}_seed{info.seed}.png")
      info.save_image(outImage, filepath)
      info.save_thumbnail(outImage, info.resize)
      """
      # デバッグ出力を保存
      rmm_filepath = os.path.join(self.output_dir, f"{timestr}_rmm_seed{info.seed}.png")
      save_image(rmm_dump, rmm_filepath, "")
      """
    
    return info

  def synchronize(self):
    torch.cuda.synchronize(device=self.pipe.device)

  def generate_grid(self, info_list, grid_info):
    timestr=time.strftime(self.timestr_format)
    filepath = os.path.join(self.output_dir, f"grid_{timestr}_seed{grid_info.seed}.png")
    single_batch_images = [Image.open(io.BytesIO(info.imageBytes)) for info in info_list]
    grid_image = create_batch_image(single_batch_images, grid_info.width, grid_info.height, grid_info.grid_column_count)
    grid_info.save_image(grid_image, filepath)
    grid_info.save_thumbnail(grid_image, grid_info.resize)

    return grid_image


# 処理モジュールのインスタンスを生成
sharedBackEnd = BackEnd()


In [ ]:
#@title 共通GUIコンポーネント

class GeneratePane:
  def __init__(self, parent):
    self.sampler_name_list = []
    self.is_image_generating = False # 画像生成中フラグ
    self.generated_file_list = []
    self.parent = parent
    self.prompt_textarea = widgets.Textarea()
    self.prompt_textarea.observe(self.on_prompt_update, names='value') #valueを監視してイベントハンドラを起動
    self.wordcount_label = widgets.Label(value=f"prompt 0/0")

    self.negative_prompt_textarea = widgets.Textarea()
    self.negative_prompt_textarea.observe(self.on_negative_prompt_update, names='value') #valueを監視してイベントハンドラを起動
    self.negative_wordcount_label = widgets.Label(value=f"negative 0/0")

    self.generate_button = widgets.Button(description="Generate")
    self.generate_button.on_click(self.parent.generate)

    self.interrupt_button = widgets.Button(description="Interrupt", disabled=True, layout=Layout(width="50%") )
    self.stop_button = widgets.Button(description="Stop", disabled=True, layout=Layout(width="50%") )

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info:GeneratedFileInfo):
    self.prompt_textarea.value = info.prompt
    self.negative_prompt_textarea.value = info.negative_prompt

  def on_prompt_update(self, change):
    word_count = sharedBackEnd.word_count(change.new)
    max_word_count = sharedBackEnd.max_word_count()
    self.wordcount_label.value = f"prompt {word_count}/{max_word_count}"

  def on_negative_prompt_update(self, change):
    word_count = sharedBackEnd.word_count(change.new)
    max_word_count = sharedBackEnd.max_word_count()
    self.negative_wordcount_label.value = f"negative {word_count}/{max_word_count}"

  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    #入力要素のレイアウトを設定
    self.prompt_textarea.placeholder="prompt"
    self.prompt_textarea.layout = Layout(width="100%",height="7em")
    self.negative_prompt_textarea.placeholder="Negative prompt"
    self.negative_prompt_textarea.layout = Layout(width="100%",height="7em", border="solid 0px")
    self.generate_button.style={"button_color":"lightskyblue"}
    #generate_stack = widgets.Stack([generate_button, interrupt_button], selected_index=0, layout = Layout(width="100%",height="100%"))
    self.generate_stack = widgets.VBox([\
      self.generate_button, widgets.HBox([self.interrupt_button, self.stop_button], layout = Layout(width="100%",height="50%")) \
    ])

    self.layout = widgets.HBox(layout=Layout(border="solid 0px"), children=[
      # 生成ボタン類
      widgets.VBox(layout=Layout(width="15%",height="auto", border="solid 0px"), children=[
        self.generate_stack,
        self.wordcount_label,
        self.negative_wordcount_label,
      ]),
      # プロンプト・ネガティブプロンプト
      widgets.VBox(layout=Layout(width="100%",height="100%", border="solid 0px"), children=[
        widgets.Box(layout = Layout(width="100%",height="100%"), children=[
          self.prompt_textarea,
        ]),
        widgets.Box(layout = Layout(width="100%",height="100%"), children=[
          self.negative_prompt_textarea,
        ]),
      ]),
    ])
    return self.layout


class SettingPane:
  def __init__(self):
    #生成パラメータ
    self.sampling_step_slider = widgets.IntSlider(min=0, max=150, value=20)
    self.sampling_method = widgets.Dropdown()
    self.CFG_scale_slider = widgets.FloatSlider(min=0, max=25.0, value=7.0)
    self.seed_input = widgets.IntText(value=-1)
    #保存画像のサイズ・枚数
    self.width_slider = widgets.IntSlider(min=8, max=2048, value=512, step=8)
    self.height_slider = widgets.IntSlider(min=8, max=2048, value=512, step=8)
    self.batch_size_input = widgets.BoundedIntText(value=1, min=1) # 1回の作業の生成枚数。枚数分だけGPUメモリを使用
    self.batch_count_input = widgets.BoundedIntText(value=6, min=1) # 生成作業を何回行うか。回数分だけ処理時間はかかるが、GPUは1回分の使用量で済む
    self.grid_column_count_input = widgets.BoundedIntText(value=3, min=1) # 画像gridの列の数

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info:GeneratedFileInfo):
    self.sampling_step_slider.value = info.step_count
    self.width_slider.value = info.width
    self.height_slider.value = info.height
    self.CFG_scale_slider.value = info.guidance_scale
    self.seed_input.value =  info.seed
    # リストにない場合は反映しない
    if info.scheduler_name in sharedBackEnd.schedulerDict.keys():
      self.sampling_method.value = info.scheduler_name

  # setupペインでのセットアップ内容を反映させる
  def set_param(self, sampler_name_list, default_width, default_height):
    self.sampler_name_list = sampler_name_list
    self.sampling_method.options=sampler_name_list
    self.sampling_method.value=sampler_name_list[0]
    self.width_slider.value = default_width
    self.height_slider.value = default_height

  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    self.sampling_step_slider.style={"description_width":"10em"}
    self.sampling_step_slider.layout=Layout(width="90%", border="solid 0px")
    self.sampling_step_slider.description="steps"
    self.sampling_method.style={"description_width":"10em"}
    self.sampling_method.layout=Layout(width="90%", border="solid 0px")
    self.sampling_method.description="method"

    self.CFG_scale_slider.style={"description_width":"10em"}
    self.CFG_scale_slider.layout=Layout(width="90%", border="solid 0px")
    self.CFG_scale_slider.description="CFG scale"
    self.seed_input.style={"description_width":"10em"}
    self.seed_input.layout=Layout(width="90%", border="solid 0px")
    self.seed_input.description="seed"

    self.width_slider.style={"description_width":"10em"}
    self.width_slider.layout=Layout(width="90%", border="solid 0px")
    self.width_slider.description="width"
    self.height_slider.style={"description_width":"10em"}
    self.height_slider.layout=Layout(width="90%", border="solid 0px")
    self.height_slider.description="height"

    self.batch_size_input.style={"description_width":"10em"}
    self.batch_size_input.layout=Layout(width="90%", border="solid 0px")
    self.batch_size_input.description="batch Size"
    self.batch_count_input.style={"description_width":"10em"}
    self.batch_count_input.layout=Layout(width="90%", border="solid 0px")
    self.batch_count_input.description="batch Count"
    self.grid_column_count_input.style={"description_width":"10em"}
    self.grid_column_count_input.layout=Layout(width="90%", border="solid 0px")
    self.grid_column_count_input.description="grid columns"

    # txt2imgの設定ペイン
    self.setting_pane_content = {
      "sampling":widgets.VBox([self.sampling_step_slider, self.sampling_method]),
      "quality params":widgets.VBox([self.CFG_scale_slider,self.seed_input]),
      "image size":widgets.VBox([
        self.width_slider, self.height_slider,
        self.batch_size_input, self.batch_count_input,
        self.grid_column_count_input
      ]),
      }
    self.setting_pane = widgets.VBox(layout = Layout(width="100%", height="100%"))
    # tabを1個ずつ追加（tab切り替えは不使用。tabの部分をタイトル表示にのみ使用）
    children_array = list(self.setting_pane_content.values())
    self.setting_pane.children = [widgets.Tab([wd]) for wd in children_array]
    for i,title in enumerate(self.setting_pane_content.keys()):
      self.setting_pane.children[i].set_title(0, title)

    self.layout = widgets.Box([self.setting_pane], layout=Layout(width="100%", border="solid 0px") )
    return self.layout

class OutputPane:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self):
    self.thumbnail_size = 64 # px
    self.thumbnail_button_width = 16 # px
    self.thumbnail_button_height = 64 # px
    # 出力画像ビューワの部分
    self.focus_image = widgets.Image(width="512", height="512")
    self.focus_image_box = widgets.Box()
    self.thumbnail_viewer = widgets.VBox()
    self.generated_file_list = []
    self.output = widgets.Output()

    #GUIデバッグ用
    self.viewer_debug = False
    #self.viewer_debug = True
    if self.viewer_debug:
      self.viewer_debug = True
      #string_generated_file_list = ["thumbnail01.png", "thumbnail02.png",]
      string_generated_file_list = ["test_image.png","test_image.png"]
      #string_generated_file_list = []
      for filename in string_generated_file_list:
        # 1回分のファイルを保存
        image = Image.open(filename)
        resize=0.25
        thumbnail = image.resize(size=(int(image.width*resize),int(image.height*resize)) )
        info = GeneratedFileInfo(
            prompt="", negative_prompt="", scheduler_name="",
            step_count=0, guidance_scale=0, seed=0,
            width=image.width, height=image.height, batch_size=0, batch_count=0, grid_column_count=0
        )
        with io.BytesIO() as buf:
          save_image(image, buf, "")
          info.imageBytes = buf.getvalue()
        with io.BytesIO() as buf:
          save_image(thumbnail, buf, "")
          info.thumbnailBytes = buf.getvalue()
        self.generated_file_list.append(info)
  
  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    self.focus_image_box.children = [self.focus_image]
    self.focus_image_box.layout = Layout(width="520", height="520") #64x64 px thumbnail
    self.focus_image.layout.object_fit = 'contain'
    self.focus_image.layout.object_position = 'top'
    self.thumbnail_viewer.layout = Layout(width="100", height="520") #64x64 px thumbnail

    self.layout = widgets.VBox(layout=Layout(border="lightgray solid 1px"), children=[
          widgets.Label(value="right click to download an image or thumbnail / click a 🖼️ button to show a full image", layout=Layout(height="2em")), # notion
          widgets.HBox([self.focus_image, self.thumbnail_viewer], layout=Layout(width="100%", height="520", border="solid 0px") ), #生成画像
          widgets.Box([self.output],layout=Layout(width="100%", height="16em", border="gray solid 1px")), # output
        ])
    return self.layout

  #"generate"ボタン押下時に画像生成し、1枚ずつサムネイル表示
  def generate_progressive(self, ui_info, pipeMethod):
    # UIと絶縁するため、ディープコピーを取る
    info = ui_info.clone_light()
    #extra_paramはシャローコピー
    info.extra_param = ui_info.extra_param

    self.output.clear_output()
    with self.output:
      #TODO: 生成中はボタンを押せないようにする
      #TODO: implement GUI of batch_size, batch_count and grid_column_count
      if self.viewer_debug:
        count = len(self.generated_file_list)
      else:
        self.generated_file_list = []
        count = info.batch_count
    
      # 先にサムネイルリストを生成
      #batch_size == 1のときはgridも生成
      if info.batch_size == 1:
        thumbnail_count = count+1
      else:
        thumbnail_count = count
      self.create_thumbnail_list(thumbnail_count)

      # 1枚ずつ生成し、その都度サムネイルを表示
      init_seed = info.seed
      count_seed = info.seed
      for i in range(count):
        # シード値が-1（自動決定）でかつ初回の生成では、画像生成側でシード値を決める
        if not(init_seed < 0 and i == 0):
          # 2回目以降またはシード値固定の場合は、すでにあるシード値に加算する
          count_seed = init_seed + i

        # 反復ごとに絶縁するため、ディープコピーを取る
        new_info = info.clone_light()
        #extra_paramはシャローコピー
        new_info.extra_param = info.extra_param

        new_info.seed = count_seed
        if self.viewer_debug:
          file = self.generated_file_list[i]
        else:
          # 画像生成
          sharedBackEnd.generate_image_progressive(new_info, pipeMethod)
          self.generated_file_list.append(new_info)
          # 画像生成時に決定したシード値を得る
          if init_seed < 0 and i == 0:
            count_seed = new_info.seed
            init_seed = new_info.seed #gridに書き込むためこれも書き換えておく
        
        if new_info.batch_size == 1:
          index = i+1 #children[0]はgrid
        else:
          index = i
        #サムネイルを表示（画像データをセット）
        self.set_thumbnail(index, new_info.thumbnailBytes)
      
    #self.output.clear_output()
    with self.output:
      #gridを作成
      if info.batch_size == 1:
        grid_info = info.clone_light()
        grid_info.extra_param = info.extra_param
        grid_info.seed = init_seed
        grid = sharedBackEnd.generate_grid(self.generated_file_list, grid_info)
        #先頭に追加
        self.generated_file_list.insert(0, grid_info)
        #サムネイルを表示（画像データをセット）
        self.set_thumbnail(0, grid_info.thumbnailBytes)
    
    self.is_image_generating = False
    return self.generated_file_list

  #先にサムネイルリストを生成
  def create_thumbnail_list(self, count):
      thumbnail_widget_list = []
      self.thumbnail_button_list = [] #クリックイベントハンドラ内でボタンに対応するインデックスを得るためのリスト
      #空のサムネイルリストを生成
      for i in range(count):
        wd = self.create_viewer_progressive()
        thumbnail_widget_list.append(wd)
      #先にサムネイルリストをビューワに追加
      self.thumbnail_viewer.children = thumbnail_widget_list

  #サムネイルに画像データをセット
  def set_thumbnail(self, index, thumbnailBytes):
    self.thumbnail_viewer.children[index].children[0].value = thumbnailBytes

  #サムネイルボタンのクリックイベントハンドラを生成
  def focus_handler(self, change):
    button = change
    i = self.thumbnail_button_list.index(button)
    self.focus_image.value = self.generated_file_list[i].imageBytes


  # 空のサムネイルを追加
  def create_viewer_progressive(self):
    image = widgets.Image(width=self.thumbnail_size, height=self.thumbnail_size)
    image.layout.object_fit = 'contain'

    # クリックしてfocus_imageを更新するボタンを生成
    button = widgets.Button(description="🖼️")
    button.layout = Layout(width=f"{self.thumbnail_button_width}", height=f"{self.thumbnail_button_height}")
    button.on_click(self.focus_handler)
    self.thumbnail_button_list.append(button)

    # 画像とボタンを並べたものを1セットとしてサムネイルに追加
    return widgets.HBox([image, button])


In [ ]:
#@title setupペイン

# モデル使用時のコンポーネント類は https://huggingface.co/fooID/barModel/
# 下の model_index.json をデフォルト設定として参照して内部的にロードされる。ほかのモデルでも同様の仕組み

class SetupUIView:
  def __init__(self):
    self.model_revision = {
      # stable diffusion official
      "stabilityai/stable-diffusion-2-1-base":{"default":"fp16", "revision":["main","bf16","fp16"]},
      "stabilityai/stable-diffusion-2-1":{"default":"fp16", "revision":["main","bf16","fp16"], "width":768, "height":768},
      #"stabilityai/stable-diffusion-2-depth":{"default":"fp16", "revision":["main","fp16"]},
      "runwayml/stable-diffusion-v1-5":{"default":"fp16", "revision":["main","bf16","flax","fp16","onnx"]},
      "naclbit/trinart_stable_diffusion_v2":{"default":"diffusers-115k", "revision":["main","diffusers-115k","diffusers-95k","diffusers-60k"]},
      "naclbit/trinart_characters_19.2m_stable_diffusion_v1":{"default":"main", "revision":["main"]},
      #"":{"default":"main", "revision":["main"]},
    }
    self.device_to_list=["cuda", "cpu"]
    self.width=512
    self.height=512

    self.model_select = widgets.Dropdown(options=list(self.model_revision.keys()), value=list(self.model_revision.keys())[0] )
    self.revision_select = widgets.Dropdown(options=[])
    self.enable_other_model_check = widgets.Checkbox(value=False)
    self.other_model_input = widgets.Text(value="")
    self.other_revision_input = widgets.Text(value="main")

    #modelに応じてrevisionリストを変える
    def update_revision(change):
      model = change['new']
      self.revision_select.options = self.model_revision[model]["revision"]
      self.revision_select.value = self.model_revision[model]["default"]
    self.model_select.observe(update_revision, 'value')
    #revision_selectの初期値を設定
    update_revision({'new':self.model_select.value})

    self.device_to_select = widgets.Dropdown(options=self.device_to_list, value=self.device_to_list[0])
    self.enable_attention_slicing_check = widgets.Checkbox(value=False)
    self.xformers_memory_efficient_attention_check = widgets.Checkbox(value=False)
    self.enable_safety_checker_check = widgets.Checkbox(value=True)

    #on_clickを持たないため、valueを監視してイベントハンドラを起動
    def on_safety_checker_check(change):
      sharedBackEnd.safety_checker = change.new    
    self.enable_safety_checker_check.observe(on_safety_checker_check, names='value')

    def on_enable_attention_slicing_check(change):
      sharedBackEnd.attention_slicing = change.new    
    self.enable_attention_slicing_check.observe(on_enable_attention_slicing_check, names='value')

    def on_xformers_memory_efficient_attention_check(change):
      sharedBackEnd.xformers_memory_efficient_attention = change.new    
    self.xformers_memory_efficient_attention_check.observe(on_xformers_memory_efficient_attention_check, names='value')

    self.setup_button = widgets.Button(description="Setup")
    self.setup_button.on_click(self.on_setup)
    self.output = widgets.Output()

  def set_layout(self):
    self.model_select.style={"description_width":"10em"}
    self.model_select.layout=Layout(width="36em")
    self.model_select.description="preset model ID"
    self.revision_select.description="revision"
    self.revision_select.layout=Layout(width="15em")
    self.enable_other_model_check.style={"description_width":"6em"}
    self.enable_other_model_check.layout=Layout(width="14em")
    self.enable_other_model_check.description="other model"
    self.other_model_input.style={"description_width":"0em"}
    self.other_model_input.layout=Layout(width="36em")
    self.other_model_input.description=""
    self.other_revision_input.style={"description_width":"4.5em"}
    self.other_revision_input.layout=Layout(width="12em")
    self.other_revision_input.description="revision"

    self.device_to_select.style={"description_width":"10em"}
    self.device_to_select.layout=Layout(width="16em")
    self.device_to_select.description="device to"
    self.enable_attention_slicing_check.style={"description_width":"10em"}
    self.enable_attention_slicing_check.layout=Layout(width="22em")
    self.enable_attention_slicing_check.description="attention slicing"
    self.xformers_memory_efficient_attention_check.style={"description_width":"10em"}
    self.xformers_memory_efficient_attention_check.layout=Layout(width="12em")
    self.xformers_memory_efficient_attention_check.description="xformers memory efficient attention"
    self.enable_safety_checker_check.style={"description_width":"10em"}
    self.enable_safety_checker_check.layout=Layout(width="22em")
    self.enable_safety_checker_check.description="safety checker"
    self.setup_button.style={"button_color":"lightgreen"}

    # setupの設定ペイン
    self.setting_pane_content = {
      "model": widgets.VBox(layout=Layout(border="solid 0px"), children=[\
        widgets.HBox(layout=Layout(border="solid 0px"), children=[ \
          self.model_select,
          self.revision_select,
        ]),
        widgets.HBox(layout=Layout(border="solid 0px"), children=[ \
          self.enable_other_model_check,
          self.other_model_input,
          self.other_revision_input,
        ]),
      ]),
      "device":widgets.VBox([
        self.device_to_select,
        widgets.HBox([
          self.enable_attention_slicing_check,
          #self.xformers_memory_efficient_attention_check,
          self.enable_safety_checker_check,
        ]),
      ]),
    }
    self.setting_pane = widgets.VBox(layout = Layout(width="62em", height="100%"))
    # tabを1個ずつ追加（tab切り替えは不使用。tabの部分をタイトル表示にのみ使用）
    children_array = list(self.setting_pane_content.values())
    self.setting_pane.children = [widgets.Tab([wd]) for wd in children_array]
    for i,title in enumerate(self.setting_pane_content.keys()):
      self.setting_pane.children[i].set_title(0, title)

    self.layout = widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
      self.setting_pane,
      self.setup_button,
      widgets.Box([self.output],layout=Layout(width="100%", height="20em", border="gray solid 1px"))
    ])

    return self.layout

  def on_setup(self, change):
    # model, revisionをチェック
    if self.enable_other_model_check.value == True:
      model = self.other_model_input.value
      revision = self.other_revision_input.value
    else:
      model = self.model_select.value
      revision = self.revision_select.value
    
    # デフォルトwidth, heightを変更
    if "width" in self.model_revision[self.model_select.value]:
      self.width = self.model_revision[self.model_select.value]["width"]
    if "height" in self.model_revision[self.model_select.value]:
      self.height = self.model_revision[self.model_select.value]["height"]
    # 画像生成モジュールをセットアップ
    self.output.clear_output()
    with self.output:
      sharedBackEnd.setup(
        model=model, revision=revision, device_to=self.device_to_select.value, 
        output_dir="/content/output", default_width=self.width, default_height=self.height,
        attention_slicing=self.enable_attention_slicing_check.value,
        xformers_memory_efficient_attention=self.xformers_memory_efficient_attention_check.value,
        safety_checker=self.enable_safety_checker_check.value,
      )

# ペインのインスタンスを生成
setupPane = SetupUIView()



In [ ]:
#@title txt2imgペイン

class Txt2imgPaneView:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self):
    self.generatePane = GeneratePane(self)
    self.settingPane = SettingPane()
    self.outputPane = OutputPane()

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info):
    self.generatePane.reflectPNGInfo(info)
    self.settingPane.reflectPNGInfo(info)

  # setupペインでのセットアップ内容を反映させる
  def set_param(self, sampler_name_list, default_width, default_height):
    self.settingPane.set_param(sampler_name_list, default_width, default_height)

  def generate(self, change):
    info = GeneratedFileInfo(
      prompt = self.generatePane.prompt_textarea.value,
      negative_prompt = self.generatePane.negative_prompt_textarea.value,
      scheduler_name = self.settingPane.sampling_method.value,
      step_count = self.settingPane.sampling_step_slider.value,
      guidance_scale = self.settingPane.CFG_scale_slider.value,
      seed = self.settingPane.seed_input.value,
      width = self.settingPane.width_slider.value,
      height = self.settingPane.height_slider.value,
      batch_size = self.settingPane.batch_size_input.value,
      batch_count = self.settingPane.batch_count_input.value,
      grid_column_count = self.settingPane.grid_column_count_input.value,
      eta=0.0,
      resize=0.25
    )
    self.outputPane.generate_progressive(info, sharedBackEnd.txt2img)



  # 入力要素の定義とレイアウト設定を分離する
  # 全体のレイアウトを設定
  def set_layout(self):
    self.layout = \
    widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
      self.generatePane.set_layout(),
      # 設定／生成画像
      widgets.HBox(layout=Layout(border="solid 0px"), children=[
        widgets.Box([self.settingPane.set_layout()], layout=Layout(width="45%")),
        widgets.Box([self.outputPane.set_layout()], layout=Layout(width="55%")),
      ]),
    ])
    return self.layout


# txt2imgペインのインスタンスを生成
txt2imgPane = Txt2imgPaneView()


In [ ]:
#@title img2imgペイン

class Img2imgPane:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self, parent):
    self.parent = parent
    #画像データ
    self.src_image = None
    #ウィジェット
    self.src_image_widget = widgets.Image(width="120", height="120")
    self.denoise_strength_slider = widgets.FloatSlider(description="Denoising strength", min=0, max=1.0, value=0.75)
    self.src_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="source img")
    self.src_upload_button.observe(self.on_src_upload, names='value') #FileUploadはon_clickを持たないため、valueを監視してイベントハンドラを起動

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info:GeneratedFileInfo):
    if 'strength' in info.extra_param:
      self.denoise_strength_slider.value = info.extra_param['strength']

  # txt2imgペインのレイアウトを設定
  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    content = widgets.VBox(layout=Layout(width="100%"), children=[
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        widgets.VBox([self.src_upload_button, self.src_image_widget]),
      ]),
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        self.denoise_strength_slider,        
      ])
    ])

    self.denoise_strength_slider.style={"description_width":"10em"}
    self.denoise_strength_slider.layout=Layout(width="20em", border="solid 0px")
    self.src_upload_button.layout=Layout(width="100")

    tab = widgets.Tab([content])
    tab.set_title(0, "img2img")

    # 全体のレイアウトを設定
    self.layout = \
    widgets.Box(layout=Layout(width="100%", border="solid 0px"), children=[
      tab,
    ])

    return self.layout

  #ファイルアップロード完了時に元画像を更新する
  def on_src_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.src_image = Image.open(io.BytesIO(data)).convert("RGB")
    self.src_image_widget.value = data
    
class Img2imgPaneView:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self):
    self.generatePane = GeneratePane(self)
    self.img2imgPane = Img2imgPane(self)
    self.settingPane = SettingPane()
    self.outputPane = OutputPane()

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info):
    self.generatePane.reflectPNGInfo(info)
    self.settingPane.reflectPNGInfo(info)
    self.img2imgPane.reflectPNGInfo(info)


  # setupペインでのセットアップ内容を反映させる
  def set_param(self, sampler_name_list, default_width, default_height):
    self.settingPane.set_param(sampler_name_list, default_width, default_height)

  def generate(self, change):
    info = GeneratedFileInfo(
      prompt = self.generatePane.prompt_textarea.value,
      negative_prompt = self.generatePane.negative_prompt_textarea.value,
      scheduler_name = self.settingPane.sampling_method.value,
      step_count = self.settingPane.sampling_step_slider.value,
      guidance_scale = self.settingPane.CFG_scale_slider.value,
      seed = self.settingPane.seed_input.value,
      width = self.settingPane.width_slider.value,
      height = self.settingPane.height_slider.value,
      batch_size = self.settingPane.batch_size_input.value,
      batch_count = self.settingPane.batch_count_input.value,
      grid_column_count = self.settingPane.grid_column_count_input.value,
      eta=0.0,
      resize=0.25,
      extra_param = {
        "strength": self.img2imgPane.denoise_strength_slider.value,
        "image": self.img2imgPane.src_image,
      }
    )
    # 画像を1枚ずつ生成
    self.outputPane.generate_progressive(info, sharedBackEnd.img2img)



  # 入力要素の定義とレイアウト設定を分離する
  # 全体のレイアウトを設定
  def set_layout(self):
    self.layout = \
    widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
      self.generatePane.set_layout(),
      # 設定／生成画像
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        widgets.VBox(layout=Layout(width="45%", border="solid 0px"), children=[
          self.img2imgPane.set_layout(),
          self.settingPane.set_layout(),
        ]),
        widgets.Box([self.outputPane.set_layout()], layout=Layout(width="50%", border="solid 0px"))
      ]),
    ])
    return self.layout



#Inpaintペインのインスタンスを生成
img2imgPane = Img2imgPaneView()


In [ ]:
#@title Inpaintペイン


class InpaintPane:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self, parent):
    self.parent = parent
    #画像データ
    self.src_image = None
    self.mask_image = None
    self.top_image = None
    self.inpaint_image = None
    #ウィジェット
    self.src_image_widget = widgets.Image(width="100", height="100")
    self.mask_image_widget = widgets.Image(width="100", height="100", layout=Layout(border="dotted 1px skyblue") )
    self.top_image_widget = widgets.Image(width="100", height="100")
    self.overlay_widget = widgets.Image(width="120", height="120")
    self.mask_switch = widgets.RadioButtons(description="Inpaint areas", value="white", options=["black", "white"])
    self.mask_switch.observe(self.on_mask_switched, names='value') #valueを監視してイベントハンドラを起動
    self.masked_content_select = widgets.Dropdown(description="Masked content", options=["fill", "original", "mode channel", "latent noise", "randomly", "top image"]) #"latent nothing"は未実装
    self.masked_content_select.observe(self.on_MaskedContent_upload, names='value') #valueを監視してイベントハンドラを起動
    self.denoise_strength_slider = widgets.FloatSlider(description="Denoising strength", min=0, max=1.0, value=0.75)
    self.src_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="back img")
    self.src_upload_button.observe(self.on_src_upload, names='value') #FileUploadはon_clickを持たないため、valueを監視してイベントハンドラを起動
    self.mask_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="mask")
    self.mask_upload_button.observe(self.on_mask_upload, names='value') #valueを監視してイベントハンドラを起動
    self.top_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="top img")
    self.top_upload_button.observe(self.on_top_upload, names='value') #valueを監視してイベントハンドラを起動

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info:GeneratedFileInfo):
    if 'latents' in info.extra_param:
      self.masked_content_select.value = info.extra_param['latents']
    if 'strength' in info.extra_param:
      self.denoise_strength_slider.value = info.extra_param['strength']

  # txt2imgペインのレイアウトを設定
  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    inpaint_content = widgets.VBox(layout=Layout(width="100%"), children=[
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        widgets.VBox([self.src_upload_button, self.src_image_widget]),
        widgets.VBox([self.mask_upload_button, self.mask_image_widget]),
        widgets.VBox([self.top_upload_button, self.top_image_widget]),
      ]),
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        self.overlay_widget,
        widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
          self.mask_switch,
          self.masked_content_select,
          self.denoise_strength_slider,        
        ])
      ])
    ])

    self.mask_switch.layout.display="flex"
    self.mask_switch.style={"description_width":"10em"}
    self.masked_content_select.style={"description_width":"10em"}
    self.masked_content_select.layout=Layout(width="95%", border="solid 0px")
    self.denoise_strength_slider.style={"description_width":"10em"}
    self.denoise_strength_slider.layout=Layout(width="95%", border="solid 0px")
    self.src_upload_button.layout=Layout(width="100")
    self.mask_upload_button.layout=Layout(width="100")
    self.top_upload_button.layout=Layout(width="100")

    inpaint_tab = widgets.Tab([inpaint_content])
    inpaint_tab.set_title(0, "inpaint")

    # 全体のレイアウトを設定
    self.layout = \
    widgets.Box(layout=Layout(width="100%", border="solid 0px"), children=[
      inpaint_tab,
    ])   

    return self.layout

  #ファイルアップロード完了時にマスクを更新する
  def on_mask_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.mask_image_widget.value = data
    self.on_mask_switched(None)
    self.overlay_mask(self.masked_content_select.value)

  #ファイルアップロード完了時にマスクを更新する
  def on_mask_switched(self, change):
    if self.mask_image_widget.value == None:
      return #何もしない
    
    mask = Image.open(io.BytesIO(self.mask_image_widget.value)).convert("RGB")
    if(self.mask_switch.value == "black"):
      #マスクを反転する
      self.mask_image = ImageChops.invert(mask)
    else:
      self.mask_image = mask
    self.overlay_mask(self.masked_content_select.value)

  #ファイルアップロード完了時に元画像を更新する
  def on_src_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.src_image = Image.open(io.BytesIO(data)).convert("RGB")
    self.src_image_widget.value = data
    self.overlay_mask(self.masked_content_select.value)

  #ファイルアップロード完了時に元画像を更新する
  def on_top_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.top_image = Image.open(io.BytesIO(data)).convert("RGB")
    self.top_image_widget.value = data
    self.overlay_mask(self.masked_content_select.value)

  #ファイルアップロード完了時に元画像を更新する
  def on_MaskedContent_upload(self, change):
    self.overlay_mask(change.new)

  # 元画像にマスクを重ねて表示
  def overlay_mask(self, latents_type):
    if self.src_image == None or self.mask_image == None:
      return #何もしない

    #マスクを元画像に重ねて表示
    self.inpaint_image = self.mask_fill_image(latents_type)

    #表示用の画像データをバッファに出力
    with io.BytesIO() as buf:
      if not (latents_type == "fill" or latents_type == "original" or latents_type == "mode channel" or latents_type == "top image"):
        # マスクを白抜きで表示
        image = self.src_image.copy()
        mask = self.mask_image.convert("L")
        image.paste(self.mask_image, mask=mask)
        image.save(buf, format="png")
      else:
        self.inpaint_image.save(buf, format="png")
      # 表示用の画像データをウィジェットに設定
      self.overlay_widget.value = buf.getvalue()

  # 元画像をマスクで切り抜いて塗りつぶす
  def mask_fill_image(self, latents_type):
    image = self.src_image.copy()
    mask = self.mask_image.convert("L")
    fill = None
    
    if latents_type == "fill":
      fill = Image.new(mode="RGB", size=(self.src_image.width, self.src_image.height), color="gray")
    if latents_type == "top image" and self.top_image != None:
      fill = self.top_image
    elif latents_type == "mode channel":
      # imageのマスク範囲におけるチャネルごとの最頻値を使う
      histo = self.src_image.histogram(mask=mask)
      ch_mode_index = [ max(rng, key=histo.__getitem__) for rng in [range(0,256), range(256,512), range(512,768)] ] #インデックスが画素値に相当
      color = (ch_mode_index[0], ch_mode_index[1]-256, ch_mode_index[2]-512) #インデックスのオフセットを引く
      print("ch_mode_index", ch_mode_index, "color", color)
      fill = Image.new(mode="RGB", size=(self.src_image.width, self.src_image.height), color=color)
    elif latents_type == "original":
      # 何もしない
      pass
    else:
      # 何もしない
      pass

    # マスクをかける画像がなければ、元の画像（背景）を返す
    if fill == None:
      return self.src_image
    else:
      # imageにマスクをかけて貼り付け
      # TODO: Mask Blurを実装
      image.paste(fill, mask=mask)
      return image

    
class InpaintPaneView:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self):
    self.generatePane = GeneratePane(self)
    self.inpaintPane = InpaintPane(self)
    self.settingPane = SettingPane()
    self.outputPane = OutputPane()

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info):
    self.generatePane.reflectPNGInfo(info)
    self.settingPane.reflectPNGInfo(info)
    self.inpaintPane.reflectPNGInfo(info)

  # setupペインでのセットアップ内容を反映させる
  def set_param(self, sampler_name_list, default_width, default_height):
    self.settingPane.set_param(sampler_name_list, default_width, default_height)

  def generate(self, change):
    info = GeneratedFileInfo(
      prompt = self.generatePane.prompt_textarea.value,
      negative_prompt = self.generatePane.negative_prompt_textarea.value,
      scheduler_name = self.settingPane.sampling_method.value,
      step_count = self.settingPane.sampling_step_slider.value,
      guidance_scale = self.settingPane.CFG_scale_slider.value,
      seed = self.settingPane.seed_input.value,
      width = self.settingPane.width_slider.value,
      height = self.settingPane.height_slider.value,
      batch_size = self.settingPane.batch_size_input.value,
      batch_count = self.settingPane.batch_count_input.value,
      grid_column_count = self.settingPane.grid_column_count_input.value,
      eta=0.0,
      resize=0.25,
      extra_param = {
        "strength": self.inpaintPane.denoise_strength_slider.value,
        "image": self.inpaintPane.inpaint_image,
        "mask_image": self.inpaintPane.mask_image,
        "latents": self.inpaintPane.masked_content_select.value,
      }
    )
    # 画像を1枚ずつ生成
    self.outputPane.generate_progressive(info, sharedBackEnd.inpaint)



  # 入力要素の定義とレイアウト設定を分離する
  # 全体のレイアウトを設定
  def set_layout(self):
    self.layout = \
    widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
      self.generatePane.set_layout(),
      # 設定／生成画像
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        widgets.VBox(layout=Layout(width="45%", border="solid 0px"), children=[
          self.inpaintPane.set_layout(),
          self.settingPane.set_layout(),
        ]),
        widgets.Box([self.outputPane.set_layout()], layout=Layout(width="50%", border="solid 0px"))
      ]),
    ])
    return self.layout



#Inpaintペインのインスタンスを生成
inpaintPane = InpaintPaneView()



In [ ]:
#@title PNGInfoペイン

# 実際のレイアウトは別途行う
class PnginfoPaneView:
  def __init__(self, txt2imgPane, img2imgPane, inpaintPane):
    self.txt2imgPane = txt2imgPane
    self.img2imgPane = img2imgPane
    self.inpaintPane = inpaintPane
    self.param_dict = {}
    self.info = None #GeneratedFileInfo

    self.pnginfo_prompt_textarea = widgets.Textarea(placeholder="prompt",layout = Layout(width="100%",height="15em"))
    
    self.pnginfo_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="upload an image")
    self.pnginfo_upload_button.observe(self.on_pnginfo_upload, names='value') #FileUploadはon_clickを持たないため、valueを監視してイベントハンドラを起動
    
    self.pnginfo_upload_image = widgets.Image(width="512", height="512")
    
    self.send_to_txt2img_button = widgets.Button(description="send to txt2img",layout = Layout(width="auto") )
    self.send_to_txt2img_button.on_click(self.on_send_to_txt2img)

    self.send_to_img2img_button = widgets.Button(description="send to img2img",layout = Layout(width="auto") )
    self.send_to_img2img_button.on_click(self.on_send_to_img2img)

    self.send_to_inpaint_button = widgets.Button(description="send to inpaint",layout = Layout(width="auto") )
    self.send_to_inpaint_button.on_click(self.on_send_to_inpaint)

    self.thumbnail_image = widgets.Image(width="64", height="64")

  def set_layout(self):
    # PNG Infoのペイン
    self.layout = \
    widgets.HBox(layout=Layout(width="100%", height="100%"), children=[\
      # 画像のアップロード
      widgets.VBox(layout=Layout(width="540", height="100%"), children=[\
        pnginfoPane.pnginfo_upload_button,
         widgets.HBox([pnginfoPane.pnginfo_upload_image], layout=Layout(width="520", height="520")),
      ] ),
      # 画像の設定パラメータ表示、SendToボタン
      widgets.VBox(layout=Layout(width="60%", height="100%"), children=[\
        widgets.HBox([pnginfoPane.pnginfo_prompt_textarea], layout=Layout(width="100%", height="100%")), \
        widgets.HBox( [\
          pnginfoPane.send_to_txt2img_button,
          pnginfoPane.send_to_img2img_button,
          pnginfoPane.send_to_inpaint_button,
        ]), \
        widgets.HBox([pnginfoPane.thumbnail_image]),
      ]), \
    ])
    return self.layout

  #ファイルアップロード完了時に画像とパラメータ表示を更新する
  def on_pnginfo_upload(self, change):
    (data, parameter_text) = update_pnginfo(new_value=change.new)
    image = Image.open(io.BytesIO(data))
    self.info = GeneratedFileInfo.from_parameter_text(parameter_text)
    self.info.imageBytes = data
    self.pnginfo_upload_image.value = data
    self.pnginfo_upload_image.width = min(512, image.width)
    self.pnginfo_upload_image.height = min(512, image.height)
    self.pnginfo_prompt_textarea.value = parameter_text
    
    # サムネイルを作成、parameter_textをそのまま保存
    resize = 0.25
    thumbnail = create_thumbnail(image=image, resize=resize)
    with io.BytesIO() as buf:
      save_image(thumbnail, buf, parameter_text)
      self.thumbnail_image.width = min(512, image.width*resize)
      self.thumbnail_image.height = min(512, image.height*resize)
      self.thumbnail_image.value = buf.getvalue()

  """
  "send to **"ボタン押下時にそれぞれのペインへ反映させる
  """
  def on_send_to_txt2img(self, remove):
    # TODO: should be event-driven
    txt2imgPane.reflectPNGInfo(self.info)

  def on_send_to_img2img(self, remove):
    # TODO: should be event-driven
    img2imgPane.reflectPNGInfo(self.info)

  def on_send_to_inpaint(self, remove):
    # TODO: should be event-driven
    inpaintPane.reflectPNGInfo(self.info)


#PNGInfoペインのインスタンスを生成
pnginfoPane = PnginfoPaneView(txt2imgPane, img2imgPane, inpaintPane)


In [ ]:
#@title 全体レイアウト

# トップメニュータブと各ペインを設定
top_tab_dict = {"setup":setupPane.set_layout(), "txt2img":txt2imgPane.set_layout(), "img2img":img2imgPane.set_layout(), "inpaint":inpaintPane.set_layout(), "PNG Info":pnginfoPane.set_layout() }
sharedBackEnd.setupPane = setupPane
sharedBackEnd.txt2imgPane = txt2imgPane
sharedBackEnd.img2imgPane = img2imgPane
sharedBackEnd.inpaintPane = inpaintPane
sharedBackEnd.pnginfoPane = pnginfoPane


top_tab = widgets.Tab(layout = Layout(width="100%",height="100%"))
top_tab.children = list(top_tab_dict.values())
for i,title in enumerate(top_tab_dict.keys()):
  top_tab.set_title(i, title)

# メインウィンドウを生成
main_window = widgets.Box([top_tab], layout = Layout(width="100%",height="100%"))

#3. 起動

In [ ]:
#@title GUIを起動。「出力を全画面表示」を推奨
main_window